In [2]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd


In [3]:
data1 = pd.read_csv("Diabetes-CLassification.csv")

In [13]:
data1

,PatientID,Pregnancies,PlasmaGlucose,DiastolicBloodPressure,TricepsThickness,SerumInsulin,BMI,DiabetesPedigree,Age,Diabetic
0,1354778,0,171,80,34,23,43.509726,1.213191,21,0
1,1147438,8,92,93,47,36,21.240576,0.158365,23,0
2,1640031,7,115,47,52,35,41.511523,0.079019,23,0
3,1883350,9,103,78,25,304,29.582192,1.282870,43,1
4,1424119,1,85,59,27,35,42.604536,0.549542,22,0
...,...,...,...,...,...,...,...,...,...,...
9995,1469198,6,95,85,37,267,18.497542,0.660240,31,0
9996,1432736,0,55,51,7,50,21.865341,0.086589,34,0
9997,1410962,5,99,59,47,67,30.774018,2.301594,43,1
9998,1958653,0,145,67,30,21,18.811861,0.789572,26,0


In [28]:
# Split the data into features (X) and labels (Y)

X = data1.drop("Diabetic", axis=1).values
y = data1["Diabetic"].values

In [30]:
#Split the data into training and testing set
X_train, y_train, X_test, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [31]:
#Define Logistic regression
class LogisticRegression(tf.keras.Model):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.dense = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
        
    def call(self, inputs):
        return self.dense(inputs)

In [32]:
#create an instance of the Logisic Regression model
model = LogisticRegression()


In [33]:
#Define the loss function
loss_object = tf.keras.losses.BinaryCrossentropy()


In [34]:
#Define the Optimizer
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

In [35]:
#define metrics for evaluation
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.BinaryAccuracy(name="train_accuracy")

In [36]:
#define the training loop

@tf.function
def train_step(inputs, labels):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

In [37]:
#perform training
num_epochs = 100
batch_size = 10
num_batches = X_train.shape[0] // batch_size

for epoch in range(num_epochs):
    train_loss.reset_states()
    train_accuracy.reset_states()
    
    for batch in range(num_batches):
        start = batch * batch_size
        end = (batch + 1) * batch_size
        batch_inputs = X_train[start:end]
        batch_labels = y_train[start:end]
        train_step(batch_inputs, batch_labels)

ValueError: in user code:

    File "C:\Users\Wanyoike\AppData\Local\Temp\ipykernel_4452\597733733.py", line 7, in train_step  *
        loss = loss_object(labels, predictions)
    File "c:\users\wanyoike\appdata\local\programs\python\python38\lib\site-packages\keras\losses.py", line 142, in __call__  **
        losses = call_fn(y_true, y_pred)
    File "c:\users\wanyoike\appdata\local\programs\python\python38\lib\site-packages\keras\losses.py", line 268, in call  **
        return ag_fn(y_true, y_pred, **self._fn_kwargs)
    File "c:\users\wanyoike\appdata\local\programs\python\python38\lib\site-packages\keras\losses.py", line 2156, in binary_crossentropy
        backend.binary_crossentropy(y_true, y_pred, from_logits=from_logits),
    File "c:\users\wanyoike\appdata\local\programs\python\python38\lib\site-packages\keras\backend.py", line 5707, in binary_crossentropy
        return tf.nn.sigmoid_cross_entropy_with_logits(

    ValueError: `logits` and `labels` must have the same shape

In [23]:
y_train

array([[1.81475200e+06, 3.00000000e+00, 1.22000000e+02, ...,
        4.02146352e+01, 2.66846118e-01, 3.10000000e+01],
       [1.93725100e+06, 7.00000000e+00, 1.28000000e+02, ...,
        2.01910198e+01, 1.37972838e-01, 2.20000000e+01],
       [1.41363200e+06, 7.00000000e+00, 7.80000000e+01, ...,
        2.72086282e+01, 1.68859030e-01, 3.20000000e+01],
       ...,
       [1.63433500e+06, 0.00000000e+00, 1.54000000e+02, ...,
        5.00860680e+01, 2.61735223e-01, 2.30000000e+01],
       [1.50287500e+06, 3.00000000e+00, 1.04000000e+02, ...,
        2.97657338e+01, 1.41686538e-01, 4.20000000e+01],
       [1.61245800e+06, 0.00000000e+00, 6.70000000e+01, ...,
        2.09277636e+01, 6.13063034e-01, 2.20000000e+01]])

In [25]:
y

0       0
1       0
2       0
3       1
4       0
       ..
9995    0
9996    0
9997    1
9998    0
9999    0
Name: Diabetic, Length: 10000, dtype: int64